In [1]:
pip install opencv-python torch ultralytics deepface moviepy numpy


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached numpy-2.0.2-cp312-cp312-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp312-cp312-win_amd64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install deepface tensorflow tf-keras


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow keras


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
# install pip moviepy
pip install moviepy 


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import torch
from deepface import DeepFace
from ultralytics import YOLO
import time
import os

# Load YOLO model
model = YOLO("yolov8n.pt")

# List of multiple reference images
reference_images = ["Virat1.jpg", "Virat2.jpg", "Virat3.jpg", "Virat4.jpg"]

# List of multiple video files
video_paths = ["VIRAT1.mp4", "VIRAT2.mp4", "VIRAT3.mp4"]

# Output folder for detected images
output_folder = "detected_faces"
os.makedirs(output_folder, exist_ok=True)

# DeepFace verification parameters
distance_metric = "cosine"  
threshold = 0.4  

# Function to process a single video
def process_video(video_path, output_folder, reference_images, model, distance_metric, threshold):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0  
    appearance_count = 0  
    timestamps = []  # Store timestamps of appearances
    detected_faces = []  # Store extracted face images
    counted_faces = set()  # Track counted faces to avoid duplicates

    # Get video FPS and calculate frame interval for 0.5s
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = fps // 2  

    # Ensure video is opened
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  

        frame_count += 1
        if frame_count % frame_interval != 0:  
            continue

        # Get current timestamp (seconds)
        timestamp = frame_count / fps  

        # Run YOLO detection
        results = model(frame)
        
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()  
            confidences = result.boxes.conf.cpu().numpy()  

            for i, (box, confidence) in enumerate(zip(boxes, confidences)):
                if confidence < 0.5:  
                    continue

                x1, y1, x2, y2 = map(int, box)

                if x2 - x1 < 30 or y2 - y1 < 30:
                    continue

                face_crop = frame[y1:y2, x1:x2]  
                face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                face_crop = cv2.resize(face_crop, (160, 160))  

                # Save detected face
                face_path = os.path.join(output_folder, f"face_{os.path.basename(video_path)}_{frame_count}_{i}.jpg")
                cv2.imwrite(face_path, face_crop)

                detected_faces.append(face_path)  # Store the face image path

                time.sleep(0.1)  

                if not os.path.exists(face_path) or os.path.getsize(face_path) == 0:
                    continue

                try:
                    for ref_img in reference_images:
                        # Convert reference image to grayscale before comparison
                        ref_gray = cv2.imread(ref_img, cv2.IMREAD_GRAYSCALE)  
                        ref_gray_path = f"gray_{ref_img}"
                        cv2.imwrite(ref_gray_path, ref_gray)

                        result = DeepFace.verify(face_path, ref_gray_path, model_name="Facenet", 
                                                enforce_detection=False, distance_metric=distance_metric)

                        if result["verified"] and result["distance"] < threshold:
                            if face_path not in counted_faces:
                                print(f"✅ Match found in {os.path.basename(video_path)} at {timestamp:.2f} seconds (frame {frame_count}), face {i}!")
                                appearance_count += 1
                                timestamps.append(timestamp)  # Store the timestamp
                                counted_faces.add(face_path)  # Mark this face as counted
                            break  

                except Exception as e:
                    print(f"Error during verification: {e}")

    # Release resources safely
    try:
        if cap.isOpened():
            cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)  
    except Exception as e:
        print(f"Error releasing resources: {e}")

    # Group timestamps into continuous presence durations
    if timestamps:
        presence_intervals = []
        start_time = timestamps[0]
        end_time = timestamps[0]

        for i in range(1, len(timestamps)):
            if timestamps[i] - end_time <= 1:  # If within 1 second, continue
                end_time = timestamps[i]
            else:
                presence_intervals.append((start_time, end_time))
                start_time = timestamps[i]
                end_time = timestamps[i]

        presence_intervals.append((start_time, end_time))  # Add last interval

        # Calculate total presence duration
        total_duration = sum(end - start for start, end in presence_intervals)

        print(f"✅ Virat appeared {appearance_count} times in {os.path.basename(video_path)}.")
        print(f"📌 Time Stamps: {timestamps}")
        print(f"⏳ Total presence duration: {total_duration:.2f} seconds")
        print("📍 Appearance Intervals:")
        for start, end in presence_intervals:
            print(f"🕒 {start:.2f}s to {end:.2f}s (Duration: {end - start:.2f}s)")
    else:
        print(f"⚠️ No presence detected in {os.path.basename(video_path)}!")

    return detected_faces

# Process each video in the list
all_detected_faces = []
for video_path in video_paths:
    print(f"Processing video: {video_path}")
    detected_faces = process_video(video_path, output_folder, reference_images, model, distance_metric, threshold)
    all_detected_faces.extend(detected_faces)

# Create a video clip from extracted images (2x Speed)
def create_summary_video(image_paths, output_path, fps=4):  # ⚡ Set FPS = 4 for 2x speed
    if len(image_paths) == 0:
        print("⚠️ No images to create summary video.")
        return
    
    first_image = cv2.imread(image_paths[0])
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is not None:
            out.write(img)

    out.release()
    print(f"🎥 Summary video saved as {output_path}")

# Create the summary video
summary_video_path = "summary_clip.mp4"
create_summary_video(all_detected_faces, summary_video_path)


0: 384x640 1 person, 106.0ms
Speed: 2.5ms preprocess, 106.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
✅ Match found at 0.50 seconds (frame 15), face 0!

0: 384x640 1 person, 1 baseball bat, 1 toothbrush, 119.2ms
Speed: 3.0ms preprocess, 119.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
✅ Match found at 1.00 seconds (frame 30), face 0!

0: 384x640 1 person, 1 baseball bat, 92.8ms
Speed: 2.0ms preprocess, 92.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
✅ Match found at 1.50 seconds (frame 45), face 0!

0: 384x640 1 person, 106.3ms
Speed: 3.7ms preprocess, 106.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 105.8ms
Speed: 1.9ms preprocess, 105.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 baseball bat, 95.6ms
Speed: 6.5ms preprocess, 95.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
✅ Match found at 3.00 s

In [3]:
import moviepy
print(moviepy.__version__)



2.1.1


In [2]:
!pip install --upgrade --force-reinstall moviepy


Defaulting to user installation because normal site-packages is not writeable
  Using cached moviepy-2.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached proglog-0.1.10-py3-none-any.whl.metadata (639 bytes)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pillow-10.4.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached moviepy-2.1.2-py3-none-any.whl (126 kB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl (31.2 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 5.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/12.6 MB 

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.3 which is incompatible.
ultralytics 8.3.80 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.2.3 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\dinka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade --force-reinstall moviepy


Defaulting to user installation because normal site-packages is not writeable
  Using cached moviepy-2.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached proglog-0.1.10-py3-none-any.whl.metadata (639 bytes)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pillow-10.4.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached moviepy-2.1.2-py3-none-any.whl (126 kB)
Using cached decorator-5.2.1-py3-none-any.whl (9.2 kB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl (31.2 MB)
Using cached

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.3 which is incompatible.
ultralytics 8.3.80 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.2.3 which is incompatible.
